In [1]:
!pip install flask

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)


In [2]:
%%writefile flsk.py
from flask import Flask, request, jsonify, render_template
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

app = Flask(__name__)

# Load and preprocess the dataset
data = pd.read_csv('Heart_Disease_Dataset.csv')  # Replace with your actual dataset path

# Preprocessing steps
binary_columns = ['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
for col in binary_columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})

# One-hot encode categorical columns
data = pd.get_dummies(data, columns=['Sex', 'AgeCategory', 'Race', 'GenHealth', 'Diabetic'], drop_first=True)

# Define features and target
X = data.drop(columns=['HeartDisease'])
y = data['HeartDisease'].map({'Yes': 1, 'No': 0})

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.form.to_dict()
    input_data = {key: float(value) for key, value in data.items()}
    input_df = pd.DataFrame([input_data], columns=X.columns)
    
    # Ensure all columns are present (fill missing columns with 0)
    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0
    
    # Reorder columns to match the model's expectations
    input_df = input_df[X.columns]
    
    prediction = clf.predict(input_df)
    result = "Yes" if prediction[0] == 1 else "No"
    
    return jsonify({'Heart Disease Prediction': result})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8501, debug=True)



Writing flsk.py


In [3]:
!python flsk.py

^C


In [5]:
!pip install flask

In [7]:
# Run this cell in Jupyter Notebook
# pandas numpy scikit-learn imbalanced-learn

import pandas as pd
import numpy as np
from flask import Flask, request, jsonify, render_template
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import threading
import webbrowser
from werkzeug.serving import run_simple

app = Flask(__name__)

# Load and preprocess the dataset
data = pd.read_csv('Heart_Disease_Dataset.csv')  # Ensure this file is in the correct path

# Preprocessing steps
binary_columns = ['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']
for col in binary_columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})

# One-hot encode categorical columns
data = pd.get_dummies(data, columns=['Sex', 'AgeCategory', 'Race', 'GenHealth', 'Diabetic'], drop_first=True)

# Define features and target
X = data.drop(columns=['HeartDisease'])
y = data['HeartDisease'].map({'Yes': 1, 'No': 0})

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

@app.route('/')
def home():
    return "Flask Server Running!"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json  # Get JSON input
    input_data = {key: float(value) for key, value in data.items()}
    input_df = pd.DataFrame([input_data], columns=X.columns)
    
    # Ensure all columns are present (fill missing columns with 0)
    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0
    
    # Reorder columns to match model expectations
    input_df = input_df[X.columns]
    
    prediction = clf.predict(input_df)
    result = "Yes" if prediction[0] == 1 else "No"
    
    return jsonify({'Heart Disease Prediction': result})

# Function to run Flask in Jupyter Notebook
def run_app():
    run_simple("0.0.0.0", 8501, app, use_reloader=False)

# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# Open browser to Flask app
webbrowser.open("http://127.0.0.1:8501/")


c:\Users\kumar\anaconda3\envs\cardeo\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8501
 * Running on http://192.168.1.4:8501
Press CTRL+C to quit


True